# **CSC 4008 Assignment 1 - 119020062 许文仲**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/drive/My Drive')

Mounted at /content/gdrive


In [ ]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
# spark = SparkSession.builder.getOrCreate()
sc = pyspark.SparkContext.getOrCreate()
path = "/content/gdrive/MyDrive/Colab Notebooks/CSC 4008/test.txt"
real_path = "/content/gdrive/MyDrive/Colab Notebooks/CSC 4008/ego-facebook(1).txt"
rdd = sc.textFile(real_path)

In [ ]:
# sc = pyspark.SparkContext.getOrCreate()
# data = sc.textFile(real_path)
# rdd = data.map(lambda x: x.split(" "))
# rdd2 = rdd.map(lambda x: (int(x[0]), int(x[1])))
# rdd5 = rdd2.map(lambda x: (x[1],x[0]))
# rdd6 = rdd2.union(rdd5)
# rdd3 = rdd6.groupByKey().map(lambda x : (x[0], list(x[1])))
# rdd4 = rdd3.sortByKey()
# rdd4.collect()

In [ ]:
# # in this cell, I test that "subtract" can be used to get the diff of first-
# # level friend RDD and the second level friend RDD
# data1 = [1,1,2,3,4]
# data2 = [1]
# rdd_data1 = sc.parallelize(data1)
# rdd_data2 = sc.parallelize(data2)
# result = rdd_data1.subtract(rdd_data2)
# type(result.collect())

list

In [ ]:

# rdd2 = rdd.map(lambda x: (int(x.split(" ")[0]), int(x.split(" ")[1])))
# rdd2_reverse = rdd.map(lambda x: (int(x.split(" ")[1]), int(x.split(" ")[0])))
# # rdd3 = rdd2.map(lambda x)
# # for element in rdd2.collect():
# #   print(element)
# rdd_union = rdd2.union(rdd2_reverse)
# # sorted(rdd_union.groupByKey().mapValues(list).collect())

In [ ]:
# get the first level friend RDD
sc = pyspark.SparkContext.getOrCreate()
data = sc.textFile(real_path)
rdd = data.map(lambda x: x.split(" "))
rdd2 = rdd.map(lambda x: (int(x[0]), int(x[1])))
rdd5 = rdd2.map(lambda x: (x[1],x[0]))
rdd6 = rdd2.union(rdd5)
rdd3 = rdd6.groupByKey().map(lambda x : (x[0], list(x[1])))
rdd4 = rdd3.sortByKey()
# rdd4.collect()

In [ ]:
# try to get the second level friend RDD
def findSecondLevelRdd(line):
  first_friend_list = line[1]
  for i in range(len(first_friend_list)):
    for j in range(i+1, len(first_friend_list)):
      yield((first_friend_list[i], first_friend_list[j]))
      yield((first_friend_list[j], first_friend_list[i]))

mapData = rdd4.flatMap(findSecondLevelRdd).groupByKey().map(lambda x : (x[0], list(x[1])))
mapData2 = mapData.sortByKey()
# mapData2.collect()[0]

In [ ]:

rdd10 = mapData2.union(rdd4)
rdd11 = rdd10.groupByKey().map(lambda x : (x[0], list(x[1])))
rdd12 = rdd11.sortByKey()


In [ ]:
# try to eliminate the first level friend
# 不懂为啥带不进去 带不进map里 后面解决了

from collections import *
def eliminateFirst(line):
  # second = line[1][0]
  second = line[0][1][0]
  # print(second)
  # first = line[1][1]
  first = line[0][1][1]
  # print(first)
  rdd_data_second = sc.parallelize(second)
  rdd_data_first = sc.parallelize(first)
  result = rdd_data_second.subtract(rdd_data_first).collect()
  result = sorted(result)
  # print(result)
  mutual = Counter(result).most_common(10)
  return (line[0][0], mutual)

# eliminateFirst(rdd12.collect()[0])

# getFriends = rdd12.map(eliminateFirst)
# getFriends.collect()

In [ ]:
# this is for whom only wants for a single index
def getResult(index):
  wanted_RDD = rdd12.filter(lambda x: x[0] == index)
# wanted_RDD.collect()[0]
  print(eliminateFirst(wanted_RDD.collect()))

wanted = [1, 10, 152, 288, 603, 714, 1525, 2434, 2681]
for i in wanted:
  getResult(i)


(1, [(710, 4), (603, 3), (714, 2), (1525, 2), (289, 1), (290, 1), (291, 1), (292, 1), (293, 1), (294, 1)])
(10, [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (11, 1), (12, 1)])
(152, [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)])
(288, [(71, 3), (1525, 3), (69, 2), (90, 2), (217, 2), (2348, 2), (2351, 2), (2352, 2), (2354, 2), (2356, 2)])
(603, [(1, 3), (289, 1), (290, 1), (291, 1), (292, 1), (293, 1), (294, 1), (295, 1), (296, 1), (297, 1)])
(714, [(1, 2), (712, 2), (713, 2), (715, 2), (717, 2), (718, 2), (1525, 2), (90, 1), (217, 1), (247, 1)])
(1525, [(288, 3), (1, 2), (710, 2), (714, 2), (603, 1)])
(2434, [(71, 2), (288, 2), (711, 2), (716, 2), (719, 2), (720, 2), (2348, 2), (2351, 2), (2352, 2), (2354, 2)])
(2681, [(71, 1), (288, 1), (710, 1), (711, 1), (716, 1), (719, 1), (720, 1), (721, 1), (722, 1), (2348, 1)])


In [ ]:
# useless
# rdd_union.groupByKey().collect()
# index = 1571
# type(rdd_union.lookup(index))
# first_level_friend_list = sorted(rdd_union.lookup(index))
# second_level_friend_list = []
# for first_level_friend in first_level_friend_list:
#   second_level_friend_list += rdd_union.lookup(first_level_friend)


In [ ]:
# 需要在 二级朋友 中 除去自身 和 一级朋友 (useless)
# second_level_friend_list = sorted(second_level_friend_list)
# # second_level_friend_set = set(second_level_friend_list)
# first_level_friend_set = set(first_level_friend_list)
# first_level_friend_set.add(index)
# # diff = second_level_friend_set - first_level_friend_set
# # diff_list = list(diff)
# # diff_list.remove(1571)
# for i in first_level_friend_set:
#   while i in second_level_friend_list:
#     second_level_friend_list.remove(i)


In [ ]:
# useless
# from collections import *
# result = Counter(second_level_friend_list).most_common(10)
# if result == []:
#   output = []
# else:
#   output = [i[0] for i in result]
# output

[710, 603, 714, 1525, 289, 290, 291, 292, 293, 294]

In [ ]:
# compute the total result by 'map' so that you can pick the one you want
from collections import *
def eliminateFirst(line):
  # second = line[1][0]
  second_level_friend_list = line[1][0]
  # print(second)
  # first = line[1][1]
  first_level_friend_list = line[1][1]
  # print(first)
  # rdd_data_second = sc.parallelize(second)
  # rdd_data_first = sc.parallelize(first)
  # result = rdd_data_second.subtract(rdd_data_first).collect()
  # second_level_friend_list = sorted(second_level_friend_list)
  # second_level_friend_set = set(second_level_friend_list)
  first_level_friend_set = set(first_level_friend_list)
  # diff = second_level_friend_set - first_level_friend_set
  # diff_list = list(diff)
  # diff_list.remove(1571)
  for i in first_level_friend_set:
    while i in second_level_friend_list:
      second_level_friend_list.remove(i)
  second_level_friend_list = sorted(second_level_friend_list)
  result = Counter(second_level_friend_list).most_common(10)

  if result == []:
    output = []
  else:
    output = result
    wrong = [i[0] for i in result]
    # print(wrong)
  # return (line[0], output)
  return (line[0], wrong)

getFriends = rdd12.map(eliminateFirst)
# getFriends.collect()

In [ ]:
# getFriends.collect()
wanted = [10, 152, 288, 603, 714, 1525, 2434, 2681]
result = getFriends.filter(lambda x: x[0] in wanted).collect()
for each in sc.parallelize(result).sortBy(lambda pair: pair[0]).collect():
    print(each)

(10, [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (11, 1), (12, 1)])
(152, [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)])
(288, [(71, 3), (1525, 3), (69, 2), (90, 2), (217, 2), (2348, 2), (2351, 2), (2352, 2), (2354, 2), (2356, 2)])
(603, [(1, 3), (289, 1), (290, 1), (291, 1), (292, 1), (293, 1), (294, 1), (295, 1), (296, 1), (297, 1)])
(714, [(1, 2), (712, 2), (713, 2), (715, 2), (717, 2), (718, 2), (1525, 2), (90, 1), (217, 1), (247, 1)])
(1525, [(288, 3), (1, 2), (710, 2), (714, 2), (603, 1)])
(2434, [(71, 2), (288, 2), (711, 2), (716, 2), (719, 2), (720, 2), (2348, 2), (2351, 2), (2352, 2), (2354, 2)])
(2681, [(71, 1), (288, 1), (710, 1), (711, 1), (716, 1), (719, 1), (720, 1), (721, 1), (722, 1), (2348, 1)])


In [ ]:
wanted = [10, 152, 288, 603, 714, 1525, 2434, 2681]
result = getFriends.filter(lambda x: x[0] in wanted).collect()
for each in sc.parallelize(result).sortBy(lambda pair: pair[0]).collect():
    print(each)

(10, [2, 3, 4, 5, 6, 7, 8, 9, 11, 12])
(152, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
(288, [71, 1525, 69, 90, 217, 2348, 2351, 2352, 2354, 2356])
(603, [1, 289, 290, 291, 292, 293, 294, 295, 296, 297])
(714, [1, 712, 713, 715, 717, 718, 1525, 90, 217, 247])
(1525, [288, 1, 710, 714, 603])
(2434, [71, 288, 711, 716, 719, 720, 2348, 2351, 2352, 2354])
(2681, [71, 288, 710, 711, 716, 719, 720, 721, 722, 2348])
